### SageMaker fine tune ChatGLM

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备requirements.txt

In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 101.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.116
    Uninstalling botocore-1.29.116:
      Successfully uninstalled botocore-1.29.116
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.116
    Uninstalling boto3-1.26.116:
      Successfully uninstalled boto3-1.26.116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.71 requires botocore==1.29.71, but you have botocore 1.29.121 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 747.7/747.7 kB 11.6

In [17]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker-us-west-2-687912291502


### chatglm 官方P-tuning v2方式（单机单卡）
1:安装依赖lib   
2:准备数据集(本例以ADGEN 文本生成数据集为例，将解压后的 AdvertiseGen 目录放到本目录  
3:修改并bash运行 train.sh  

In [7]:
#!pip install rouge_chinese nltk jieba datasets
#!git clone https://github.com/THUDM/ChatGLM-6B.git
!pip install -r ChatGLM-6B/requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 132.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [29]:
#!cd ChatGLM-6B/ptuning/ && wget "https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1"
#!cd ChatGLM-6B/ptuning/ && mv "index.html?dl=1" dataset.tar.gz
#!cd ChatGLM-6B/ptuning/ && tar -xvf dataset.tar.gz
!./ChatGLM-6B/ptuning/s5cmd sync ChatGLM-6B/ptuning/AdvertiseGen/ s3://{bucket}/llm/chatglm/datasets/ 
#!rm -rf cd ChatGLM-6B/ptuning/dataset.tar.gz


cp ChatGLM-6B/ptuning/AdvertiseGen/dev.json s3://sagemaker-us-west-2-687912291502/llm/chatglm/datasets/dev.json
cp ChatGLM-6B/ptuning/AdvertiseGen/train.json s3://sagemaker-us-west-2-687912291502/llm/chatglm/datasets/train.json


In [3]:
!cd ChatGLM-6B/ptuning/&& bash train.sh

04/27/2023 09:11:17 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
04/27/2023 09:11:17 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
generation_max_len

### chatglm 官方deepspeed方式（全参数的Finetune,单机多卡）
1: 准备deepspeed lib，并修改deepspeed.json    
2：数据集（以上一致）  
3：entrypoint start.py,设置deepspeed hosts configure
4：触发bash ds_train_finetune.sh 

In [21]:
#print('s3://{}/llm/models/'.format(sagemaker_session.default_bucket()))
#!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/
!curl -L https://github.com/peak/s5cmd/releases/download/v2.0.0/s5cmd_2.0.0_Linux-64bit.tar.gz | tar -xz && mv s5cmd ChatGLM-6B/ptuning/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4176k  100 4176k    0     0  14.7M      0 --:--:-- --:--:-- --:--:-- 14.7M


In [23]:
instance_type = 'ml.p4d.24xlarge'
if instance_type in [
    "ml.p3.16xlarge",
    "ml.p3dn.24xlarge",
    "ml.g5.48xlarge",
    "ml.p4d.24xlarge",
]:
    processes_per_host = 8
elif instance_type == "ml.p2.16xlarge":
    processes_per_host = 16
else:
    processes_per_host = 4

print("processes_per_host is set to:", processes_per_host)

processes_per_host is set to: 8


In [25]:
# define Training Job Name 
job_name = f'huggingface-chatglm-deepspeed-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/deepspeed/'.format(sagemaker_session.default_bucket())
output_dir = '/opt/ml/model/adgen-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'


instance_count = 2
#define the enviroment variables for your scripts.
environment = {
              'MODEL_S3_PATH'          : model_s3_path,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'NUM_GPUS'               : processes_per_host，
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path
}

inputs={
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}


# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'start.py',          # user endpoint script
    source_dir           = 'ChatGLM-6B/ptuning',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment,
)
huggingface_estimator.fit(inputs=inputs)